# Set Up Environment

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weed25-yolov8/README.dataset.txt
/kaggle/input/weed25-yolov8/README.roboflow.txt
/kaggle/input/weed25-yolov8/data.yaml
/kaggle/input/weed25-yolov8/valid/labels/barnyard-grass-115-_JPG.rf.b9a9909b97f0c27d5b641550b8a58e8b.txt
/kaggle/input/weed25-yolov8/valid/labels/Indian_aster-261-_JPG.rf.de8d944f7ca4899124a35adc18ab94b2.txt
/kaggle/input/weed25-yolov8/valid/labels/cocklebur-267-_JPG.rf.bae6648953b81b7d0786e9e421684696.txt
/kaggle/input/weed25-yolov8/valid/labels/Asiatic-Smartweed-415-_JPG.rf.d38584ac41b503a46467cfa4acbfbfbb.txt
/kaggle/input/weed25-yolov8/valid/labels/Ceylon-spinach-386-_JPG.rf.c62d6575cb4ab924f901a9fbeaeef3a6.txt
/kaggle/input/weed25-yolov8/valid/labels/cocklebur-278-_JPG.rf.bf9eba79b9833d50b2e020330c5be8bc.txt
/kaggle/input/weed25-yolov8/valid/labels/cocklebur-363-_JPG.rf.bf35b215f1be3ae3ee0d2c2859928ebf.txt
/kaggle/input/weed25-yolov8/valid/labels/Indian_aster-32-_JPG.rf.594e54fd57a89bdfabecdf6068d36223.txt
/kaggle/input/weed25-yolov8/valid/labels/IMG

In [2]:
!pip install --quiet ultralytics
# # from ultralytics import YOLO
# # model = YOLO('yolov8n.pt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 15.8 MB/s eta 0:00:00


In [3]:
!pip install learn2learn --quiet
# !pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.1/178.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.0 MB/s eta 0:00:00
   

In [4]:
# Uploading the dataset
data_dir = "/kaggle/input/new-data-weed/weed detection.v1i.coco/train"
data_ann = "/kaggle/input/new-data-weed/weed detection.v1i.coco/train/_annotations.coco.json"

In [5]:
import os
import shutil
from pycocotools.coco import COCO

# Load the dataset
coco = COCO(data_ann)
output_dir = "/kaggle/working/weed25_fewshot"
img_dir = data_dir

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Create a folder for each category
categories = coco.loadCats(coco.getCatIds())
category_id_to_name = {cat['id']: cat['name'].replace(" ", "_") for cat in categories}

for class_name in category_id_to_name.values():
    os.makedirs(os.path.join(output_dir, class_name), exist_ok=True)

# Track copied images to avoid duplicates
copied_images = set()

# Group images by category and copy them
for ann in coco.dataset['annotations']:
    img_id = ann['image_id']
    cat_id = ann['category_id']
    class_name = category_id_to_name[cat_id]
    
    img_info = coco.loadImgs(img_id)[0]
    img_file = img_info['file_name']
    src = os.path.join(img_dir, img_file)
    dst = os.path.join(output_dir, class_name, img_file)
    
    if dst not in copied_images and os.path.exists(src):
        shutil.copy(src, dst)
        copied_images.add(dst)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
for root, dirs, files in os.walk(output_dir):
    print(root, "->", len(files), "files")

/kaggle/working/weed25_fewshot -> 0 files
/kaggle/working/weed25_fewshot/weed -> 25 files


In [7]:
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os

class WeedDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.images = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith(".jpg")]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 1  # All are weeds

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(),
    transforms.ToTensor()
])

dataset = WeedDataset("/kaggle/working/weed25_fewshot/weed", transform=transform)


In [9]:
import learn2learn as l2l
import torch.nn as nn
import torch.nn.functional as F

class ProtoNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten()
        )
        
    def forward(self, x):
        return self.encoder(x)


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ProtoNetEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten()
        )

    def forward(self, x):
        return self.encoder(x)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def create_episode(dataset, k_shot=5, q_queries=5):
    # Random shuffle
    indices = torch.randperm(len(dataset))
    support_idxs = indices[:k_shot]
    query_idxs = indices[k_shot:k_shot + q_queries]

    support_set = torch.stack([dataset[i][0] for i in support_idxs])
    support_labels = torch.tensor([dataset[i][1] for i in support_idxs])

    query_set = torch.stack([dataset[i][0] for i in query_idxs])
    query_labels = torch.tensor([dataset[i][1] for i in query_idxs])

    return support_set, support_labels, query_set, query_labels


In [12]:
def compute_prototypes(support, labels):
    """
    support: [n_support, feature_dim]
    labels: [n_support]
    """
    classes = torch.unique(labels)
    prototypes = []
    for cls in classes:
        cls_mask = labels == cls
        cls_embeddings = support[cls_mask]
        prototype = cls_embeddings.mean(dim=0)
        prototypes.append(prototype)
    return torch.stack(prototypes), classes


In [13]:
def euclidean_dist(a, b):
    # a: [n_query, feature_dim], b: [n_class, feature_dim]
    n = a.size(0)
    m = b.size(0)
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    return torch.pow(a - b, 2).sum(dim=2)


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time


In [15]:
# Fake: duplicate weed images and treat some as "background"
from torchvision import transforms

# Example: Create "weed" and "background" by augmenting
weed_imgs = [dataset[i][0] for i in range(len(dataset))]
weed_labels = [0] * len(weed_imgs)

background_imgs = [transforms.RandomInvert()(img) for img in weed_imgs]  # or blur, rotate, etc.
background_labels = [1] * len(background_imgs)

# Combine
all_imgs = weed_imgs + background_imgs
all_labels = weed_labels + background_labels


In [16]:
import random

def create_episode(dataset, n_classes=2, k_shot=5, q_queries=5):
    """
    Ensure each episode samples `n_classes` distinct classes
    """
    labels = list(set([label for _, label in dataset]))
    selected_classes = random.sample(labels, n_classes)

    support_set = []
    query_set = []

    for cls in selected_classes:
        cls_items = [(img, label) for img, label in dataset if label == cls]
        random.shuffle(cls_items)
        support_set += cls_items[:k_shot]
        query_set += cls_items[k_shot:k_shot + q_queries]

    support_x = torch.stack([x[0] for x in support_set])
    support_y = torch.tensor([x[1] for x in support_set])
    query_x = torch.stack([x[0] for x in query_set])
    query_y = torch.tensor([x[1] for x in query_set])

    return support_x, support_y, query_x, query_y


In [17]:
from torch.utils.data import Dataset

class WeedBackgroundDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

dataset = WeedBackgroundDataset(all_imgs, all_labels)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ProtoNetEncoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(30):
    model.train()

    start_time = time.time()

    support_x, support_y, query_x, query_y = create_episode(dataset)

    support_x = support_x.to(device)
    query_x = query_x.to(device)
    support_y = support_y.to(device)
    query_y = query_y.to(device)

    support_embeddings = model(support_x)
    query_embeddings = model(query_x)

    prototypes, class_ids = compute_prototypes(support_embeddings, support_y)
    dists = euclidean_dist(query_embeddings, prototypes)

    # Convert labels to prototype indices
    label_indices = torch.zeros_like(query_y)
    for i, cls in enumerate(class_ids):
        label_indices[query_y == cls] = i

    loss = F.cross_entropy(-dists, label_indices)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    preds = torch.argmax(-dists, dim=1).cpu()
    true = label_indices.cpu()

    acc = accuracy_score(true, preds)
    precision = precision_score(true, preds, average='macro', zero_division=0)
    recall = recall_score(true, preds, average='macro', zero_division=0)
    f1 = f1_score(true, preds, average='macro', zero_division=0)

    epoch_time = time.time() - start_time

    print(f"Epoch {epoch+1:02d} | Loss: {loss.item():.4f} | "
          f"Acc: {acc*100:.2f}% | Prec: {precision:.2f} | Rec: {recall:.2f} | F1: {f1:.2f} | "
          f"Time: {epoch_time:.2f}s")


Epoch 01 | Loss: 2.0656 | Acc: 40.00% | Prec: 0.22 | Rec: 0.40 | F1: 0.29 | Time: 2.30s
Epoch 02 | Loss: 0.8059 | Acc: 60.00% | Prec: 0.60 | Rec: 0.60 | F1: 0.60 | Time: 2.36s
Epoch 03 | Loss: 0.6813 | Acc: 50.00% | Prec: 0.50 | Rec: 0.50 | F1: 0.49 | Time: 1.81s
Epoch 04 | Loss: 0.6948 | Acc: 60.00% | Prec: 0.62 | Rec: 0.60 | F1: 0.58 | Time: 1.82s
Epoch 05 | Loss: 1.3412 | Acc: 20.00% | Prec: 0.20 | Rec: 0.20 | F1: 0.20 | Time: 1.82s
Epoch 06 | Loss: 0.7313 | Acc: 50.00% | Prec: 0.50 | Rec: 0.50 | F1: 0.49 | Time: 1.85s
Epoch 07 | Loss: 0.6553 | Acc: 60.00% | Prec: 0.78 | Rec: 0.60 | F1: 0.52 | Time: 1.81s
Epoch 08 | Loss: 0.7012 | Acc: 50.00% | Prec: 0.50 | Rec: 0.50 | F1: 0.49 | Time: 1.77s
Epoch 09 | Loss: 0.6887 | Acc: 60.00% | Prec: 0.62 | Rec: 0.60 | F1: 0.58 | Time: 1.73s
Epoch 10 | Loss: 0.6779 | Acc: 50.00% | Prec: 0.50 | Rec: 0.50 | F1: 0.49 | Time: 1.82s
Epoch 11 | Loss: 0.7104 | Acc: 60.00% | Prec: 0.60 | Rec: 0.60 | F1: 0.60 | Time: 1.79s
Epoch 12 | Loss: 0.6054 | Acc: 7